Итоговое задание **Максима Андрийчука**

по **Проекту 5**. Компьютер говорит «Нет»

# Содержание 

* [Информация](#section-one)
* [Описание данных](#section-two)
* [Визуализация данных](#section-three)
* [Тест данных на наивной модели](#section-four)
* [EDA](#section-five)
    - [Числовые признаки](#subsection-one)
    - [Категориальные признаки](#subsection-two)
    - [Бинарные признаки](#subsection-three)
* [Feature engineering](#section-six)
* [Стандартизация числовых признаков](#section-seven)
* [Оценка значимости переменных](#section-eight)
* [Создаем Get_dummies с категориальных признаков](#section-nine)
* [Обучение и тест моделей](#section-ten)
    - [Логистическая модель на дизбалансированных даннных](#subsection-four)
    - [Логистическая модель на сбалансированных данных](#subsection-five)
* [Практикуем Гиперпараметры](#section-twelve)
    - [Логистическая модель с гиперпараметрами](#subsection-seven)
* [Попробуем модель KNN (К-Ближайших Соседей)](#section-fourteen)
* [Попробуем модель DecisionTreeClassifier](#section-fifteen)
* [Submission](#section-sixteen)

In [162]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from pandas import Series
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score

from pylab import rcParams

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [163]:
# Отрисовать ROC кривую, где AUC означает площадь под кривой (area under curve) 
def calc_and_plot_roc(y_valid, probs):
    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()

<a id="section-one"></a>
## Информация

In [164]:
path = '/kaggle/input/sf-dst-scoring/'

In [165]:
train = pd.read_csv(path +'/train.csv')
test = pd.read_csv(path +'test.csv')
sample = pd.read_csv(path +'/sample_submission.csv')

In [166]:
display(train.info())
print('Размерность тренировочного датасета: ', train.shape)
display(train.head(2))
print('')
print('=================================================')
print('')
display(test.info())
print('Размерность тестового датасета: ', test.shape)
display(test.head(2))

In [167]:
# # загружаем датасет
# df_train = pd.read_csv("/kaggle/input/sf-dst-scoring/train.csv")
# df_test = pd.read_csv("/kaggle/input/sf-dst-scoring/test.csv")
# sample_submission = pd.read_csv("/kaggle/input/sf-dst-scoring/sample_submission.csv")

In [168]:
# посмотрим какое распределение классов в целевой переменной
display(round(train['default'].value_counts(1)*100, 1))
display(sns.countplot(x='default', data=train))

Видно, что целевой признак имеет дизбаланс классов 87/13

In [169]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1   # train
test['sample'] = 0    # test
test['default'] = 0  # imaginary value for now
data = train.append(test, sort=False).reset_index(drop=True)
data.info()

In [170]:
# Посмотрим в каких признакаx есть пропуски
data.isna().sum().nlargest()

<a id="section-two"></a>
## Описание данных

Резюме по предварительному анализу: в тренировочной выборке 73799 клиентов, в тестовой - 36349. Всего данные о 110148 клиентах. Всего 20 переменных, из них 1 - временной ряд, 6 бинарных, 7 категориальных и 8 числовых. Всего пропусков 478 (0.4%), все пропуски в переменной education. client_id уникальный числовой признак, который не несет полезностей. В бинарных признаках наше целевая переменная default и искуственно добавленный признак тренировочной части датасета sample

client_id - идентификатор клиента

education - уровень образования

sex - пол заемщика

age - возраст заемщика

car - флаг наличия автомобиля

car_type - флаг автомобиля иномарки

decline_app_cnt - количество отказанных прошлых заявок

good_work - флаг наличия “хорошей” работы

bki_request_cnt - количество запросов в БКИ

home_address - категоризатор домашнего адреса

work_address - категоризатор рабочего адреса

income - доход заемщика

foreign_passport - наличие загранпаспорта

sna - связь заемщика с клиентами банка

first_time - давность наличия информации о заемщике

score_bki - скоринговый балл по данным из БКИ

region_rating - рейтинг региона

app_date - дата подачи заявки

default - флаг дефолта по кредиту

<a id="section-three"></a>
## Визуализация данных

In [171]:
data.sample(5)

In [172]:
# А теперь заполним пропуски в поле 'education'
#mode_ed = data['education'].mode().iloc[0]
data['education'].fillna('UNKW', inplace=True)

<a id="section-four"></a>
## Тест данных на наивной модели

In [173]:
%%time
# plt.rcParams['figure.figsize'] = (10,10)
# Посмотрим какой ROC AUC модель покажет на сырых данных
data_try = data[data.corr().columns]
X = data_try.drop(['default'], axis = 1)#.values
Y = data_try['default']#.values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

# обучаем модель
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)
# получаем предказания
y_pred = model.predict(X_test)
lr_probs = model.predict_proba(X_test)
# сохраняем вероятности только для положительного исхода
lr_probs = lr_probs[:, 1]
# рассчитываем ROC AUC
lr_auc = roc_auc_score(y_test, lr_probs)
print('LogisticRegression: ROC AUC=%.3f' % (lr_auc))


# Посчитать значения ROC кривой и значение площади под кривой AUC
fpr, tpr, threshold = roc_curve(y_test, lr_probs)
roc_auc = roc_auc_score(y_test, lr_probs)

# строим график
calc_and_plot_roc(y_test, lr_probs)

In [174]:
# Считаем confusion_matrix
confusion_matrix(y_test, y_pred)

<a id="section-five"></a>
## EDA

In [175]:
# посмотрим количество уникальных значений в каждом признаке
for col in data.columns:
    print(col, ':', data[col].nunique(), sep='')

In [176]:
# внесем данные из резюме в списки 
# временной ряд
time_cols = ['app_date']
# числовые переменные
num_cols = ['age','decline_app_cnt','score_bki','income','bki_request_cnt','region_rating'] 
# категориальные переменные
cat_cols = ['education','work_address','home_address','sna','first_time']
# бинарные переменные 
bin_cols = ['sex','car','car_type','good_work','foreign_passport'] 

<a id="subsection-one"></a>
## Числовые признаки

In [177]:
# Посмотрим графики распределения числовых призаков одним кодом
fig, axes = plt.subplots(2, 3, figsize=(25,15))

for i,col in enumerate(num_cols):
    sns.distplot(data[col], kde=True, ax=axes.flat[i], color="g")

# data.hist(figsize=(25, 15))
# plt.tight_layout

Видим, что распределение совсем не похоже на нормальное...Попробуем поработать с логарифмическими величинами наших переменных.

Логарифмируем только те столбцы, распределение которых совсем не похоже на нормальное:

In [178]:
# посмотрим распределение логарифмированных числовых признаков
fig, axes = plt.subplots(1, 3, figsize=(10,7))
for i,col in enumerate(['decline_app_cnt', 'bki_request_cnt', 'income']):
    data[col] = np.log(data[col] + 1)
    sns.distplot(data[col][data[col] > 0].dropna(), ax=axes.flat[i], kde = True, rug=False, color="g")

Столбцы теперь выглядят лучше, но выбросы все равно есть.

In [179]:
# Теперь построим боксплоты для наших числовых переменных:
def boxplot(col):
    fig, axes = plt.subplots(figsize = (14, 4))
    sns.boxplot(x='default', y=col, data=data[data['sample']==1],ax=axes)
    axes.set_title('Boxplot for ' + col)
    plt.show()
for col in num_cols:
    boxplot(col)

In [180]:
# # Построим boxplot’ы для численных переменных
# for z in range(len(num_cols)):
#     sns.boxplot(x="default", y=num_cols[z], data=data)
#     plt.show()



1) Недефолтные клиенты, как правило, старше.

2) Скоринговый балл выше у дефолтных клиентов

3) В среднем, доход недефолтных клиентов незначительно выше, чем у дефолтных.

4) Рейттинг региона значительно влияет на положительный ответ к запросу о выдаче кредита.

5) Количество запросов в БКИ в среднем больше у недефотных клиентов.


С выбросами разберемся позже.

In [181]:
# Оценим корреляцию Пирсона для числовых переменных
# rcParams['figure.figsize'] = 10, 5
sns.heatmap(data[num_cols].corr().abs(), annot=True, fmt=".2f", linewidths=0.1, vmin=0, vmax=1, cmap="RdBu")

<a id="subsection-two"></a>
## Категориальные признаки

In [182]:
# # Можно перевести числовые приздаки 'age' и 'income' в категориальные если потребуется
# data['income'] = pd.qcut(data['income'],
#                               q=[0, .25, .5, .75, 1],
#                               labels=['1', '2', '3', '4'])
# data['age'] = pd.qcut(data['age'],
#                               q=[0, .25, .5, .75, 1],
#                               labels=['1', '2', '3', '4'])

In [183]:
# Смотрим зависимость балл по БКИ от рейтинга региона
# plt.figure(figsize=(15, 8))
sns.boxplot(x="region_rating", y="score_bki", data=data, showfliers=False)

Рейтинг региона пусть и незначительно, но влияет на балл по БКИ.

In [184]:
#  Смотрим зависимость дохода от возраста
plt.figure(figsize=(15, 8))
sns.boxplot(x="age", y="income", data=data, showfliers=False)

In [185]:
# Смотрим зависимость дохода от образования
# plt.figure(figsize=(15, 8))
sns.boxplot(x="education", y="income", data=data, showfliers=False)

Чем выше уровень образования и возраст, в среднем тем больше доход.

In [186]:
# # график распределений всех категорий признака 'education'
# year  = data['education'].value_counts()
# plt.figure(figsize=(5,5))
# sns.barplot(year.index, year.values, alpha=0.9)
# plt.xticks(rotation=90)

# # data['education'].value_counts().plot.barh()

В основе метода оценки значимости переменных лежит однофакторный дисперсионный анализ (ANOVA). Основу процедуры составляет обобщение результатов двух выборочных t-тестов для независимых выборок (2-sample t). 


В качестве меры значимости мы будем использовать значение f-статистики. Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для нашей линейной модели.

<a id="subsection-three"></a>
## Бинарные признаки

In [187]:
# Посмотрим зависимость дохода от хорошей работы
# plt.figure(figsize=(15, 8))
sns.boxplot(x="good_work", y="income", data=data, showfliers=False)

In [188]:
# Смотрим зависимость дохода от образования
# plt.figure(figsize=(15, 8))
sns.boxplot(x="education", y="income", data=data, showfliers=False)

<a id="section-six"></a>
## Feature engineering

In [189]:
# Начнем, пожалуй, со столбца с датами
data['app_date'] = pd.to_datetime(data.app_date)

In [190]:
data['app_date'].sample(3)

Теперь можем посчитать количество дней, прошедших с самой первой даты в нашем датасете.

In [191]:
data['app_date'] = data['app_date'].apply(lambda x: (x - data['app_date'].min()).days) 

In [192]:
# Запомним наш датасет на всякий случай
df = data.copy()

In [193]:
# средний доход по возрасту
mean_income = df.groupby('age')['income'].mean().to_dict()
df['mean_income_age'] = df['age'].map(mean_income)

In [194]:
# Теперь найдем максимальный доход по возрасту
max_income = df.groupby('age')['income'].max().to_dict()
df['max_income_age'] = df['age'].map(max_income)

In [195]:
# Нормализуем
df["normalized_income"] = abs((df.income - df.mean_income_age)/df.max_income_age)

In [196]:
# Найдем среднее количество запросов в БКИ в зависимости от дохода и возраста
mean_bki = df.groupby('age')['bki_request_cnt'].mean().to_dict()
df['mean_requests_age'] = df['age'].map(mean_bki)

In [197]:
mean_bki_inc = df.groupby('income')['bki_request_cnt'].mean().to_dict()
df['mean_requests_income'] = df['income'].map(mean_bki_inc)

In [198]:
# Найдем средний уровень дохода в зависимости от рейтинга региона
mean_income_rat = df.groupby('region_rating')['income'].mean().to_dict()
df['mean_income_region'] = df['region_rating'].map(mean_income_rat)

In [199]:
# Создадим несколько новых признаков из 'app_date'
# data['app_date_day'] = data.app_date.dt.weekday
# data['app_date_week'] = data.app_date.dt.week
# data['app_date_month'] = data.app_date.dt.month
# data['timestamp'] = data.app_date.values.astype(np.int64) // 10 ** 9
#data['timestamp'] = data[['app_date']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)

In [200]:
# Создадим признак 'сколько зарегистрировано клиентов в день'
# data2 = data.copy()
# data2.index = data.app_date
# data3 = data2.resample('D').client_id.nunique()
# data4 = pd.DataFrame(data=data3)
# data4 = data4.rename(columns={'client_id': 'clients_today'})
# data = pd.merge(left=data,
#          right=data4,
#          on="app_date")

Теперь трансформируем наши бинарные и категориальные признаки с помощью LabelEncoder и OneHotEncoder.¶

In [201]:
# Для бинарных признаков мы будем использовать LabelEncoder
mapped_data_bin = {}
label_encoder = LabelEncoder()
for col in bin_cols:
    df[col] = label_encoder.fit_transform(df[col])
    mapped_data_bin[col] = dict(enumerate(label_encoder.classes_))
    

# # или так
# label_encoder = LabelEncoder()

# for column in bin_cols:
#     data[column] = label_encoder.fit_transform(data[column])
#
# label_encoder.categories_

In [202]:
mapped_data_bin

Для того, чтобы мы смогли применять линейные модели на таких данных, нам необходим методOne-Hot Encoding. Смысл его точно такой же, как у dummy-кодирования.

In [203]:
mapped_data_cat = {}
enc = OneHotEncoder()
for col in cat_cols:
    df[col] = label_encoder.fit_transform(df[col])
    mapped_data_cat[col] = dict(enumerate(label_encoder.classes_))
    


In [204]:
mapped_data_cat

In [205]:
# # Категориальные переменные можно превратить в цифра по категориям
# encoder = LabelEncoder()
# for z in cat_cols:
#     data[z] = data[z].astype('category')
#     data[z] = encoder.fit_transform(data[z])

Переобозначим наши колонки

In [206]:
df.columns

In [207]:
num_cols = ['age','decline_app_cnt','score_bki','income','bki_request_cnt','app_date', 'mean_income_age','region_rating','max_income_age', 'normalized_income',
       'mean_requests_age', 'mean_requests_income', 'mean_income_region'] # Числовые переменные
cat_cols = ['education','work_address','home_address','sna','first_time'] # категориальные переменные
bin_cols = ['sex','car','car_type','good_work','foreign_passport'] # бинарные переменные

Найдем границы выбросов

In [208]:
for col in num_cols:
    median = df[col].median()
    IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
    perc25 = df[col].quantile(0.25)
    perc75 = df[col].quantile(0.75)
    print("Колонка ", col)
    print('25-й перцентиль: {},'.format(perc25), '75-й перцентиль: {},'.format(perc75), 
          "IQR: {}, ".format(IQR),"Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

In [209]:
df.age.hist(color='r', figsize=(10,6))

В age выбросов нет

In [210]:
df.decline_app_cnt.hist(color="orange",figsize=(10,6))

в decline_app_cnt выбросов нет, если не считать Границы выбросов: [0.0, 0.0]

In [211]:
df.bki_request_cnt.hist(color="g",figsize=(10,6))

У bki_request_cnt выбросов нет.


In [212]:
df.mean_income_age.hist(color="b",figsize=(10,6))

In [213]:
df.region_rating.hist(color="purple",figsize=(10,6))

In [214]:
# # функция удаляет все строки с выбросами, но тут явных выбросов не обнаружено
# def remove_outliers(dataframe):
#     l = len(dataframe.columns)
#     for i in range (0,l,1):
#         Q1 = dataframe[dataframe.columns[i]].quantile(0.25)
#         Q3 = dataframe[dataframe.columns[i]].quantile(0.75)
#         IQR = Q3-Q1
#         dataframe = dataframe[(dataframe[dataframe.columns[i]] >= Q1 - 1.5 * IQR) & (dataframe[dataframe.columns[i]] <= Q3 + IQR* 1.5)]
#     return dataframe

# remove_outliers(data)

<a id="section-seven"></a>
## Стандартизация числовых признаков

In [215]:
dataset = df.copy() # Еще раз сохраняем наш датасет

In [216]:
dataset.head(3)

In [217]:
# Стандартизация числовых переменных
dataset[num_cols] = StandardScaler().fit_transform(df[num_cols])

# # или так
#dataset[num_cols] = pd.DataFrame(StandardScaler().fit_transform(df[num_cols]), columns = df[num_cols].columns)

<a id="section-eight"></a>
## Оценка значимости переменных

**Числовые переменные**

In [218]:
data_temp = dataset.loc[data['sample'] == 1] # обучающая выборка

In [219]:
rcParams['figure.figsize'] = 15, 8
imp_num = pd.Series(f_classif(data_temp[num_cols], data_temp['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

**Как можно увидеть, наибольшее влияние на "дефолтность" оказывает признак score_bki. Из всех сгенерированных признаков хоть какую-то значимость привносит признак mean_income_region.**

In [220]:
# Оценим корреляцию Пирсона для числовых переменных
sns.heatmap(data_temp[num_cols].corr().abs(), annot=True, fmt=".2f", linewidths=0.1, vmin=0, vmax=1, cmap="RdBu")

**Категориальные и Бинарные переменные**

Для оценки значимости категориальных и бинарных переменных будем использовать функцию mutual_info_classif из библиотеки sklearn. Данная функция опирается на непараметрические методы, основанные на оценке энтропии в группах категориальных переменных.

In [221]:
imp_cat = pd.Series(mutual_info_classif(data_temp[bin_cols + cat_cols], data_temp['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

**Итак, здесь самым значимым является признак sna(связь заемщика с клиентами), а затем признаки first_time, home_address и education.**

In [222]:
# Оценим корреляцию Пирсона для бинарных и категориальных переменных
# rcParams['figure.figsize'] = 20, 15
sns.heatmap(data_temp[bin_cols+cat_cols].corr().abs(), annot=True, fmt=".2f", linewidths=0.1, vmin=0, vmax=1, cmap="RdBu")

**Претенденты на исключение: car и car_type, work_address и home_address, first_time и sna, app_date_week и app_date_month**

<a id="section-nine"></a>
## Создаем Get_dummies с категориальных признаков

In [223]:
df = pd.get_dummies(dataset, prefix=cat_cols, columns=cat_cols) # dummy для категориальных признаков

In [224]:
data = df.copy()

In [225]:
data.head(3)

**И объединим стандартизованные числовые, бинарные и закодированные категориальные переменные в одно признаковое пространство, разделив при этом признаки и целевую переменную.**

In [226]:
# # Или все можно было сделать так:
# # Применим OneHotEncoder для категориальных признаков
# X_cat = OneHotEncoder(sparse = False).fit_transform(data[cat_cols].values)
#
# # Стандартизация числовых переменных
# X_num = StandardScaler().fit_transform(data[num_cols].values)
#
# # Объединяем все в один массив
# X = np.hstack([X_num, data[bin_cols].values, X_cat])
# Y = data['default'].values

In [227]:
# # Если нужно, удалим ненужную переменную и избавимся от признаков, которые сильно коррелируют с другими
# data = data.drop(['client_id', 'car', 'home_address', 'first_time'], axis = 1)

<a id="section-ten"></a>
## Обучение и тест моделей

<a id="subsection-four"></a>
### Логистическая модель на дизбалансированных даннных

In [228]:
# Теперь выделим тестовую часть
train_df = df.query('sample == 1').drop(['sample','client_id'], axis=1)
test_df = df.query('sample == 0').drop(['sample','client_id'], axis=1)
test_df = test_df.drop(['default'], axis = 1)

In [229]:
# Выведем топ 10 признаков коррелирующих с default
dct = {col: data[[col, 'default']].corr().iloc[0, 1] for col in data.columns[:-1]}
pd.Series(dct).apply(abs).nlargest(5)

In [230]:
X = train_df.drop(['default'], axis=1).values
y = train_df['default'].values # целевая переменная

In [231]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

In [232]:
# Обучаем Логистическаю регрессию 
model = LogisticRegression(max_iter = 1000) # 

In [233]:
# Сделаем предсказание с помощью обученой модели
model.fit(X_train, y_train)

y_pred = model.predict(X_valid)

probs = model.predict_proba(X_valid)
probs = probs[:,1]

**Кривая ROC AUC**

In [234]:
# Посчитать значения ROC кривой и значение площади под кривой AUC
rcParams['figure.figsize'] = 8, 4
fpr, tpr, threshold = roc_curve(y_valid, probs)
roc_auc = roc_auc_score(y_valid, probs)

# Отрисовать ROC кривую, где AUC означает площадь под кривой (area under curve) 
calc_and_plot_roc(y_valid, probs)

In [235]:
print('ROC AUC: %.4f' % roc_auc)
print('accuracy_score: %.4f' % accuracy_score(y_valid,y_pred))
print('precision_score: %.4f' % precision_score(y_valid,y_pred))
print('recall_score: %.4f' % recall_score(y_valid,y_pred))
print('f1_score: %.4f' % f1_score(y_valid,y_pred))

**Судя по метрикам, модель достаточно плохо справляется...**

**Confusion matrix**

In [236]:
# Визуализация confusion matrix:
sns.set_context(context='paper', font_scale=2, rc=None)
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                confusion_matrix(y_valid, y_pred).flatten()]
labels = [f'{v1}\n{v2}' for v1, v2 in
          zip(group_names, group_counts)]
labels = np.asarray(labels).reshape(2, 2)
sns.heatmap(confusion_matrix(y_valid, y_pred), annot=labels, fmt='', cmap='Blues')

In [237]:
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# cm = confusion_matrix(y_valid, y_pred)
# cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
# cmd.plot()
# cmd.ax_.set(xlabel='Predicted', ylabel='True')

Недефолтных клиентов модель предсказывает хорошо: 12872 из 14665, т.е. 87.8 %, однако, дефолтных клиентов модель предсказывает не так хорошо: 42 из 95, т.е. 44.2%, что мне кажется плохим результатом. Получается, наша модель будет выдавать кредит практически всем клиентам....Возможно, это связано с несбалансированностью нашей выборки. Попробуем модель на сбалансированных данных и потом обучить ее же с гиперпараметрами.

<a id="subsection-five"></a>
## Логистическая модель на сбалансированных данных

In [238]:
%%time
# сделаем выборку сбалансированной с помощью undersampling
n = train_df.default.value_counts().loc[1] # узнаем количество строк в классе меньшенстве
# разделим датасет на класс большинства и класс меншинства
vis_data_maj = train_df[train_df.default == 0]
vis_data_min = train_df[train_df.default == 1]
# обрежем класс большинства до количества класса меньшинства
vis_data_maj = vis_data_maj[:n]
# обьеденим 2 датасета с одинаковым количеством выборок по обоим классам
# после undersampling сначала должен идти класс большинство, а затем класс меньшинство
# Из-за того, что мы не делает shuffle в данном случае это критично
vis_data = vis_data_maj.append(vis_data_min)

X_ = vis_data.drop(['default'], axis = 1)
Y_ = vis_data['default']
X_train, X_val, Y_train, Y_val = train_test_split(X_, Y_, test_size = 0.3, random_state=42) # shuffle=False

model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)

y_pred_bal = model.predict(X_val)

# сохраняем вероятности только для положительного исхода
lr_probs_bal = model.predict_proba(X_val)
lr_probs_bal = lr_probs_bal[:, 1]

print('Метрики модели с дизбалансированными данными')
print('')
print('ROC AUC: %.4f' % roc_auc)
print('accuracy_score: %.4f' % accuracy_score(y_valid,y_pred))
print('precision_score: %.4f' % precision_score(y_valid,y_pred))
print('recall_score: %.4f' % recall_score(y_valid,y_pred))
print('f1_score: %.4f' % f1_score(y_valid,y_pred))
print('------------------------------------')
print('Метрики модели со сбалансированными данными')
print('')
print('ROC AUC: %.4f' % roc_auc_score(Y_val, lr_probs_bal))
print('accuracy_score: %.4f' % accuracy_score(Y_val,y_pred_bal))
print('precision_score: %.4f' % precision_score(Y_val,y_pred_bal))
print('recall_score: %.4f' % recall_score(Y_val,y_pred_bal))
print('f1_score: %.4f' % f1_score(Y_val,y_pred_bal))

**При сбалансированных данных, мы потеряли немного в ROC AUC, но зато существенно улучшили другие метрики**

In [239]:
# Визуализация confusion matrix на сбалансированной выборке:
sns.set_context(context='paper', font_scale=2, rc=None)
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                confusion_matrix(Y_val, y_pred_bal).flatten()]
labels = [f'{v1}\n{v2}' for v1, v2 in
          zip(group_names, group_counts)]
labels = np.asarray(labels).reshape(2, 2)
sns.heatmap(confusion_matrix(Y_val, y_pred_bal), annot=labels, fmt='', cmap='Blues')

<a id="section-twelve"></a>
## Практикуем Гиперпараметры

In [240]:
# рассмотрим поиск гиперпараметров GridSearchCV при помощи перебора параметров по сетке с последующей кросс-валидацией
from sklearn.model_selection import GridSearchCV

# Добавим типы регуляризации
penalty = ['l1', 'l2']

# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)

# Создадим гиперпараметры
hyperparameters = dict(C=C, penalty=penalty)

model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)

best_model = clf.fit(X_train, Y_train)

# View best hyperparameters
print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

In [241]:
# Более подробные варианты гиперпараметров можно увидеть тут
iter_ = 15
epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]

gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5, verbose=0)
gridsearch.fit(X_train, Y_train)
model = gridsearch.best_estimator_

best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

preds = model.predict(X_val)
print('ROC AUC: %.4f'% roc_auc_score(Y_val, lr_probs_bal))
print('Accuracy: %.4f' % accuracy_score(Y_val, preds))
print('Precision: %.4f' % precision_score(Y_val, preds))
print('Recall: %.4f' % recall_score(Y_val, preds))
print('F1: %.4f' % f1_score(Y_val, preds))

Можно использовать RandomizedSearchCV. Помог данный сайт: http://espressocode.top/ml-hyperparameter-tuning/

<a id="subsection-seven"></a>
## Логистическая модель с гиперпараметрами

In [242]:
%%time
# сделаем выборку сбалансированной с помощью undersampling
n = train_df.default.value_counts().loc[1] # узнаем количество строк в классе меньшенстве
# разделим датасет на класс большинства и класс меншинства
vis_data_maj = train_df[train_df.default == 0]
vis_data_min = train_df[train_df.default == 1]
# обрежем класс большинства до количества класса меньшинства
vis_data_maj = vis_data_maj[:n]
# обьеденим 2 датасета с одинаковым количеством выборок по обоим классам
# после undersampling сначала должен идти класс большинство, а затем класс меньшинство
# Из-за того, что мы не делает shuffle в данном случае это критично
vis_data = vis_data_maj.append(vis_data_min)

X_hyp = vis_data.drop(['default'], axis = 1)
Y_hyp = vis_data['default']
X_train_hyp, X_val_hyp, Y_train_hyp, Y_val_hyp = train_test_split(
    X_hyp, Y_hyp, test_size = 0.3, random_state=42) # shuffle=False

model_hyp = LogisticRegression(solver='liblinear', C=1.0,
                              max_iter=1000, penalty='l1' ,
                              tol=0.001) # , multi_class='ovr'
model_hyp.fit(X_train_hyp, Y_train_hyp)

y_pred_bal_hyp = model_hyp.predict(X_val_hyp)

# сохраняем вероятности только для положительного исхода
lr_probs_bal_hyp = model_hyp.predict_proba(X_val_hyp)
lr_probs_bal_hyp = lr_probs_bal_hyp[:, 1]

print('Метрики модели без гиперпараметров')
print('')
print('ROC AUC: %.4f' % roc_auc_score(Y_val, lr_probs_bal))
print('accuracy_score: %.4f' % accuracy_score(Y_val,y_pred_bal))
print('precision_score: %.4f' % precision_score(Y_val,y_pred_bal))
print('recall_score: %.4f' % recall_score(Y_val,y_pred_bal))
print('f1_score: %.4f' % f1_score(Y_val,y_pred_bal))
print('------------------------------------')
print('')
print('Метрики модели с гиперпараметрами')
print('')
print('ROC AUC: %.4f' % roc_auc_score(Y_val_hyp, lr_probs_bal_hyp))
print('accuracy_score: %.4f' % accuracy_score(Y_val_hyp,y_pred_bal_hyp))
print('precision_score: %.4f' % precision_score(Y_val_hyp,y_pred_bal_hyp))
print('recall_score: %.4f' % recall_score(Y_val_hyp,y_pred_bal_hyp))
print('f1_score: %.4f' % f1_score(Y_val_hyp,y_pred_bal_hyp))

**Логистическая модель с гиперпараметрами улучшила все метрики**

<a id="section-fourteen"></a>
## Попробуем модель KNN (К-Ближайших Соседей)

kNN — один из простейших алгоритмов классификации, поэтому на реальных задачах он зачастую оказывается неэффективным. Помимо точности классификации, проблемой этого классификатора является скорость классификации. Но все равно попробуем его для сравнения.

In [243]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold

# Попробуем вторую модель KNN для сравнения:
knn_model=KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train_hyp, Y_train_hyp)
# получаем предказания
y_pred2 = knn_model.predict_proba(X_val_hyp)
# сохраняем вероятности только для положительного исхода
y_pred2 = y_pred2[:, 1]
# рассчитываем ROC AUC
print('Модель KNN')
print('')
print('Roc_Auc_Score: ', np.round(roc_auc_score(Y_val_hyp, y_pred2),4))

# cross_val_score с KNN
kf = KFold(n_splits=10)
answer = cross_val_score(knn_model, X_train_hyp, Y_train_hyp, cv=kf, scoring="accuracy")
print('Cross_Validation_Score: ', round(np.mean(answer), 4))


###############################################
# cross_val_score с наше прошлой Логистической регрессией
answer = cross_val_score(model_hyp, X_train_hyp, Y_train_hyp, cv=kf, scoring="accuracy")
print('----------')
print('')
print('Модель Логистической регрессии с гиперпараметрами')
print('')
print('ROC AUC: %.4f' % roc_auc_score(Y_val_hyp, lr_probs_bal_hyp))
print('Cross_Validation_Score: ', round(np.mean(answer), 4))

**К-Ближайших Соседей модели результат ниже, чем с Логистической регрессией**

<a id="section-fifteen"></a>
## Попробуем модель DecisionTreeClassifier без гиперпараметров, а потом с ними

In [244]:
from sklearn.tree import DecisionTreeClassifier
# разделяем его на 2 выборки
X_train_hyp, X_val_hyp, Y_train_hyp, Y_val_hyp = train_test_split(
    X_hyp, Y_hyp, test_size = 0.3, random_state=42) # shuffle=False

tree_model = DecisionTreeClassifier(random_state=0)
tree_model.fit(X_train_hyp, Y_train_hyp)
Y_pred = tree_model.predict(X_val_hyp)

# сохраняем вероятности только для положительного исхода
Y_pred_prob = tree_model.predict_proba(X_val_hyp)
Y_pred_prob = Y_pred_prob[:, 1]

print('Метрики модели DecisionTreeClassifier')
print('')
print('ROC AUC: %.4f' % roc_auc_score(Y_val_hyp, Y_pred_prob))
print('accuracy_score: %.4f' % accuracy_score(Y_val_hyp,Y_pred))
print('precision_score: %.4f' % precision_score(Y_val_hyp,Y_pred))
print('recall_score: %.4f' % recall_score(Y_val_hyp,Y_pred))
print('f1_score: %.4f' % f1_score(Y_val_hyp,Y_pred))

**Эти результаты хуже, чем Логистическая модель с гиперпараметрами.**

**Попробуем и сюда добавить гиперпараметров, для начала узнаем оптимальные с помощью RandomizedSearchCV**

In [245]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {"max_depth": [3, None],

              "max_features": randint(1, 9),

              "min_samples_leaf": randint(1, 9),

              "criterion": ["gini", "entropy"]}

tree = DecisionTreeClassifier() # дерево решений

# Создание объекта RandomizedSearchCV
tree_cv = RandomizedSearchCV(tree, param_dist, cv = 5)

model_2 = tree_cv.fit(X_train_hyp, Y_train_hyp)
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Результат: {}".format(tree_cv.best_score_))

In [246]:
# Обучаем с параметрами
tree = DecisionTreeClassifier(criterion='entropy',max_depth=3, max_features=8, min_samples_leaf=1)
model_2 = tree.fit(X_train_hyp, Y_train_hyp)
pred = model_2.predict(X_val_hyp)
probs = model_2.predict_proba(X_val_hyp)
probs = probs[:,1]

# Кривая ROC AUC
calc_and_plot_roc(Y_val_hyp, probs)

In [247]:
print('Метрики модели DecisionTreeClassifier')
print('')
print('ROC AUC: %.4f' % roc_auc_score(Y_val_hyp, Y_pred_prob))
print('accuracy_score: %.4f' % accuracy_score(Y_val_hyp,Y_pred))
print('precision_score: %.4f' % precision_score(Y_val_hyp,Y_pred))
print('recall_score: %.4f' % recall_score(Y_val_hyp,Y_pred))
print('f1_score: %.4f' % f1_score(Y_val_hyp,Y_pred))
print('---------------------------------')
print('')
print('Метрики DecisionTreeClassifier с гиперпараметрами')
print('')
print('ROC AUC: %.4f' % roc_auc_score(Y_val_hyp, probs))
print('accuracy_score: %.4f' % accuracy_score(Y_val_hyp,pred))
print('precision_score: %.4f' % precision_score(Y_val_hyp,pred))
print('recall_score: %.4f' % recall_score(Y_val_hyp,pred))
print('f1_score: %.4f' % f1_score(Y_val_hyp,pred))

**Все же Логистическая модель с гиперпараметрами и результатом** ROC AUC: 0.7378 **лучше чем все рассмотренные выше. Будем использовать ее для наших предсказаний вероятности дефолта клиента.**

<a id="section-sixteen"></a>
## Submission

In [248]:
test_df.columns

In [249]:
%%time

# получаем предказания
y_pred = model_hyp.predict(test_df)
predict_submission = model_hyp.predict_proba(test_df)
# сохраняем вероятности только для положительного исхода
predict_submission = predict_submission[:, 1]
predict_submission


In [250]:
submission = test[['client_id']]
submission['default'] = predict_submission
submission.head(10)

In [251]:
submission.to_csv('submission.csv', index=False)

In [252]:
# Надо попробовать
# pip install lazypredict

In [253]:
# # importing LazyClassifier for classification problem because here we are solving Classification use case.
# from lazypredict.Supervised import LazyClassifier

# clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
# models,prediction = clf.fit(X_train_hyp, X_val_hyp, Y_train_hyp, Y_val_hyp)

# print(models)